In [1]:
import os
import shutil
import random

In [2]:
seed = 1

In [1]:
directory = 'C:\\Users\\Joe\\Desktop\\Data Science Masters\\Independent\\Tensorflow\\Skin Cancer Project\\Project\\Images\\'
train = 'data\\train\\'
test = 'data\\test\\'
validation = 'data\\validation\\'

In [4]:
'''
random.seed(seed)

os.makedirs(train + 'benign\\')
os.makedirs(train + 'malignant\\')
os.makedirs(test + 'benign\\')
os.makedirs(test + 'malignant\\')
os.makedirs(validation + 'benign\\')
os.makedirs(validation + 'malignant\\')


test_examples = train_examples = validation_examples = 0

for line in open('labels.csv').readlines()[1:]:
    split_line = line.split(',')
    img_file = split_line[0]
    benign_malign = split_line[1]

    random_num = random.random()

    if random_num< 0.8:
        location = train
        train_examples += 1
    elif random_num < 0.9:
        location = validation
        validation_examples += 1
    else:
        location = test
        test_examples += 1

    if int(float(benign_malign)) == 0:
        shutil.copy(
            directory + img_file + '.jpg',
            location + 'benign\\' + img_file + '.jpg'
        )
    elif int(float(benign_malign)) == 1:
        shutil.copy(
            directory + img_file + '.jpg',
            location + 'malignant\\' + img_file + '.jpg'
        )
print(f'Number of training examples: {train_examples}')
print(f'Number of validation examples {validation_examples}')
print(f'Number of test examples {test_examples}')
'''

"\nrandom.seed(seed)\n\nos.makedirs(train + 'benign\\')\nos.makedirs(train + 'malignant\\')\nos.makedirs(test + 'benign\\')\nos.makedirs(test + 'malignant\\')\nos.makedirs(validation + 'benign\\')\nos.makedirs(validation + 'malignant\\')\n\n\ntest_examples = train_examples = validation_examples = 0\n\nfor line in open('labels.csv').readlines()[1:]:\n    split_line = line.split(',')\n    img_file = split_line[0]\n    benign_malign = split_line[1]\n\n    random_num = random.random()\n\n    if random_num< 0.8:\n        location = train\n        train_examples += 1\n    elif random_num < 0.9:\n        location = validation\n        validation_examples += 1\n    else:\n        location = test\n        test_examples += 1\n\n    if int(float(benign_malign)) == 0:\n        shutil.copy(\n            directory + img_file + '.jpg',\n            location + 'benign\\' + img_file + '.jpg'\n        )\n    elif int(float(benign_malign)) == 1:\n        shutil.copy(\n            directory + img_file + '

In [5]:
import tensorflow as tf
import math
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import layers
from sklearn.metrics import roc_auc_score


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
from tensorflow import float32

In [8]:
train_examples = 20225
test_examples = 2555
validation_examples = 2551

In [9]:
img_height = img_width = 224

In [10]:
batch_size = 16

In [20]:
model = keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1', trainable=True),
    layers.Dense(128, activation = 'relu', kernel_regularizer = keras.regularizers.L2(l2=0.01)),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(1, activation = 'sigmoid')
]
)

In [21]:
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 15,
    zoom_range = (0.95, 0.95),
    horizontal_flip = True,
    vertical_flip = True,
    data_format = 'channels_last',
    #dtype = float32

)

In [22]:
validation_datagen = ImageDataGenerator(
    rescale = 1.0/255, #dtype = float32
    )

In [23]:
test_datagen = ImageDataGenerator(rescale = 1.0/255, #dtype = float32
)

In [24]:
train_gen = train_datagen.flow_from_directory(
    'data\\train\\', target_size = (img_height, img_width),
    batch_size = batch_size, color_mode = 'rgb', class_mode = 'binary', shuffle=True, seed=123
)

validation_gen = validation_datagen.flow_from_directory(
    'data\\validation\\', target_size = (img_height, img_width),
    batch_size = batch_size, color_mode = 'rgb', class_mode = 'binary', shuffle=True, seed=123
)

test_gen = test_datagen.flow_from_directory(
    'data\\test\\', target_size = (img_height, img_width),
    batch_size = batch_size, color_mode = 'rgb', class_mode = 'binary', shuffle=True, seed=123
)

Found 20225 images belonging to 2 classes.
Found 2551 images belonging to 2 classes.
Found 2555 images belonging to 2 classes.


In [25]:
METRICS = [
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
]

In [26]:
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=3e-4),
    loss = keras.losses.BinaryCrossentropy(from_logits=False),
    metrics = METRICS
)

In [35]:
model.fit(train_gen, epochs = 8, steps_per_epoch=train_examples//batch_size, validation_data=validation_gen, validation_steps = validation_examples//batch_size,
callbacks=[keras.callbacks.ModelCheckpoint('efficientnet_model_beyond_twelve', save_best_only=False, monitor='val_auc')])

Epoch 1/8
1264/1264 [==============================] - ETA: 0s - loss: 0.2951 - accuracy: 0.9346 - precision: 0.8409 - recall: 0.7841 - auc: 0.9672INFO:tensorflow:Assets written to: efficientnet_model_beyond_twelve\assets


INFO:tensorflow:Assets written to: efficientnet_model_beyond_twelve\assets


1264/1264 [==============================] - 524s 414ms/step - loss: 0.2951 - accuracy: 0.9346 - precision: 0.8409 - recall: 0.7841 - auc: 0.9672 - val_loss: 0.4019 - val_accuracy: 0.9037 - val_precision: 0.7692 - val_recall: 0.6349 - val_auc: 0.9154
Epoch 2/8
1264/1264 [==============================] - ETA: 0s - loss: 0.2878 - accuracy: 0.9357 - precision: 0.8417 - recall: 0.7902 - auc: 0.9696INFO:tensorflow:Assets written to: efficientnet_model_beyond_twelve\assets


INFO:tensorflow:Assets written to: efficientnet_model_beyond_twelve\assets


1264/1264 [==============================] - 493s 390ms/step - loss: 0.2878 - accuracy: 0.9357 - precision: 0.8417 - recall: 0.7902 - auc: 0.9696 - val_loss: 0.4057 - val_accuracy: 0.8931 - val_precision: 0.7179 - val_recall: 0.6335 - val_auc: 0.9076
Epoch 3/8
1264/1264 [==============================] - ETA: 0s - loss: 0.2787 - accuracy: 0.9416 - precision: 0.8671 - recall: 0.7966 - auc: 0.9722INFO:tensorflow:Assets written to: efficientnet_model_beyond_twelve\assets


...

1264/1264 [==============================] - 568s 449ms/step - loss: 0.2520 - accuracy: 0.9503 - precision: 0.8810 - recall: 0.8357 - auc: 0.9795 - val_loss: 0.4552 - val_accuracy: 0.8789 - val_precision: 0.6400 - val_recall: 0.6893 - val_auc: 0.8924
Epoch 7/8
1264/1264 [==============================] - ETA: 0s - loss: 0.2478 - accuracy: 0.9530 - precision: 0.8874 - recall: 0.8453 - auc: 0.9806INFO:tensorflow:Assets written to: efficientnet_model_beyond_twelve\assets


INFO:tensorflow:Assets written to: efficientnet_model_beyond_twelve\assets


1264/1264 [==============================] - 591s 468ms/step - loss: 0.2478 - accuracy: 0.9530 - precision: 0.8874 - recall: 0.8453 - auc: 0.9806 - val_loss: 0.4533 - val_accuracy: 0.8986 - val_precision: 0.7191 - val_recall: 0.6765 - val_auc: 0.9007
Epoch 8/8
1264/1264 [==============================] - ETA: 0s - loss: 0.2408 - accuracy: 0.9543 - precision: 0.8865 - recall: 0.8550 - auc: 0.9825INFO:tensorflow:Assets written to: efficientnet_model_beyond_twelve\assets


INFO:tensorflow:Assets written to: efficientnet_model_beyond_twelve\assets


1264/1264 [==============================] - 616s 487ms/step - loss: 0.2408 - accuracy: 0.9543 - precision: 0.8865 - recall: 0.8550 - auc: 0.9825 - val_loss: 0.4062 - val_accuracy: 0.9025 - val_precision: 0.7526 - val_recall: 0.6538 - val_auc: 0.9082


In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              4049564   
                                                                 
 dense_3 (Dense)             (None, 128)               163968    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 32)                4128      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,217,693
Trainable params: 4,175,677
No

In [34]:
#model.evaluate(test_gen) after twelve epochs:

160/160 [==============================] - 34s 208ms/step - loss: 0.5642 - accuracy: 0.8008 - precision: 0.4651 - recall: 0.8559 - auc: 0.9124


[0.5642102360725403,
 0.8007827997207642,
 0.4650602340698242,
 0.8558758497238159,
 0.9124125838279724]

In [36]:
model.evaluate(test_gen)

160/160 [==============================] - 37s 227ms/step - loss: 0.4245 - accuracy: 0.8892 - precision: 0.6900 - recall: 0.6763 - auc: 0.9044


[0.4244541823863983,
 0.8892368078231812,
 0.6900452375411987,
 0.6762749552726746,
 0.9043944478034973]

In [39]:
model.save(filepath='twenty_epoch_full_model')

INFO:tensorflow:Assets written to: twenty_epoch_full_model\assets


INFO:tensorflow:Assets written to: twenty_epoch_full_model\assets
